## notebook to load .ptseries parcels and export them as tsv for the feature visualizer

In [27]:
import nibabel as nb
import nilearn as nl
import hcp_utils as hcp
import numpy as np
import pandas as pd
import glob

## First average the files together

In [ ]:
#which files exist?
asc_flist = open("hbn_asc_50.txt",'r').read().splitlines()
nt_flist = open("hbn_nt_50.txt",'r').read().splitlines()

asc_exists=[]
nt_exists=[]

n_asc=0
n_nt=0
print('----- extracting ASCs -----')
for f in asc_flist:
    try:
        im_file = glob.glob(f'../sourcedata/data/HBN/brain/clean/{f}/ses-*/{f}_ses-*_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii')
        #print(im_file[0])
        img = nb.load(im_file[0])
#         img_y = img.get_fdata()
#         #img_y = scaler.fit_transform(img_y)
        asc_exists.append(im_file[0])
        print(f'exists  {f}')
        n_asc=n_asc+1
    except:
        print(f'file for {f} dne')
        
print('----- extracting NTs -----')
for f in nt_flist:
    try:
        im_file = glob.glob(f'../sourcedata/data/HBN/brain/clean/{f}/ses-*/{f}_ses-*_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii')
        #print(im_file[0])
        img = nb.load(im_file[0])
#         img_y = img.get_fdata()
#         #img_y = scaler.fit_transform(img_y)
        nt_exists.append(im_file[0])
        print(f'exists  {f}')
        n_nt=n_nt+1

    except:
        print(f'file for {f} dne')

In [64]:
asc_command="wb_command -cifti-average ../sourcedata/data/HBN/brain/clean/sub-all/sub-all_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii"
for asc in asc_exists:
    asc_command=asc_command+" -cifti "
    asc_command=asc_command+asc

nt_command="wb_command -cifti-average ../sourcedata/data/HBN/brain/clean/sub-all/sub-all_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii"
for nt in nt_exists:
    nt_command=nt_command+" -cifti "
    nt_command=nt_command+nt


In [66]:
nt_command

'wb_command -cifti-average ../sourcedata/data/HBN/brain/clean/sub-all/sub-all_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/clean/sub-NDARAC904DMU/ses-HBNsiteRU/sub-NDARAC904DMU_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/clean/sub-NDARAE012DGA/ses-HBNsiteRU/sub-NDARAE012DGA_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/clean/sub-NDARAL828WXM/ses-HBNsiteCBIC/sub-NDARAL828WXM_ses-HBNsiteCBIC_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/clean/sub-NDARAV610EY3/ses-HBNsiteRU/sub-NDARAV610EY3_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/clean/sub-NDARBU928LV0/ses-HBNsiteRU/sub-NDARBU928LV0_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii -cifti ../sourcedata/data/HBN/brain/cl

In [41]:
parcel_file='../sourcedata/data/HBN/brain/clean/sub-NDARAC904DMU/ses-HBNsiteRU/sub-NDARAC904DMU_ses-HBNsiteRU_task-movieDM_space-fsLR_den-91k_bold_clean_mmp.ptseries.nii'

im_file = f'{parcel_file}'
img = nb.load(im_file)
img_y = img.get_fdata()
#img_y = scaler.fit_transform(img_y)

#HBN TR is usually 800ms
TR=0.8

In [48]:
for i in np.arange(parcel_number)[:2]:
    label=hcp.mmp['labels'][i+1] #odd that 0 is nothing '' ??? why?
    timecourse=img_y[:,i]
    onset=np.arange(timecourse.shape[0])*TR
    duration=np.ones(timecourse.shape[0])*TR
    data=np.column_stack((onset,duration,timecourse))
    print(data)
    df=pd.DataFrame(data,columns=['onset','duration',label])
    df.to_csv(f'../outputs/parcel_timecourse_tsv/HBN/sub-NDARAC904DMU/{label}.tsv', index=False, sep="\t")

[[ 0.00000000e+00  8.00000000e-01 -3.37082863e+00]
 [ 8.00000000e-01  8.00000000e-01 -2.07178800e-05]
 [ 1.60000000e+00  8.00000000e-01 -2.07178800e-05]
 ...
 [ 5.97600000e+02  8.00000000e-01 -2.06976365e-05]
 [ 5.98400000e+02  8.00000000e-01 -2.06976365e-05]
 [ 5.99200000e+02  8.00000000e-01 -2.07178800e-05]]
[[ 0.00000000e+00  8.00000000e-01 -5.99727011e+00]
 [ 8.00000000e-01  8.00000000e-01 -1.40176460e-04]
 [ 1.60000000e+00  8.00000000e-01 -1.40176460e-04]
 ...
 [ 5.97600000e+02  8.00000000e-01 -1.40176460e-04]
 [ 5.98400000e+02  8.00000000e-01 -1.40176460e-04]
 [ 5.99200000e+02  8.00000000e-01 -1.40176460e-04]]


In [8]:
timecourse[17:].shape

(1013,)

In [13]:
hcp.mmp['labels'][198]
timecourse=img_y[:,i]

'R_FFC'

In [7]:
img_y.shape

(1030, 360)

In [9]:
hcp.mmp['labels']

{0: '',
 1: 'L_V1',
 2: 'L_MST',
 3: 'L_V6',
 4: 'L_V2',
 5: 'L_V3',
 6: 'L_V4',
 7: 'L_V8',
 8: 'L_4',
 9: 'L_3b',
 10: 'L_FEF',
 11: 'L_PEF',
 12: 'L_55b',
 13: 'L_V3A',
 14: 'L_RSC',
 15: 'L_POS2',
 16: 'L_V7',
 17: 'L_IPS1',
 18: 'L_FFC',
 19: 'L_V3B',
 20: 'L_LO1',
 21: 'L_LO2',
 22: 'L_PIT',
 23: 'L_MT',
 24: 'L_A1',
 25: 'L_PSL',
 26: 'L_SFL',
 27: 'L_PCV',
 28: 'L_STV',
 29: 'L_7Pm',
 30: 'L_7m',
 31: 'L_POS1',
 32: 'L_23d',
 33: 'L_v23ab',
 34: 'L_d23ab',
 35: 'L_31pv',
 36: 'L_5m',
 37: 'L_5mv',
 38: 'L_23c',
 39: 'L_5L',
 40: 'L_24dd',
 41: 'L_24dv',
 42: 'L_7AL',
 43: 'L_SCEF',
 44: 'L_6ma',
 45: 'L_7Am',
 46: 'L_7PL',
 47: 'L_7PC',
 48: 'L_LIPv',
 49: 'L_VIP',
 50: 'L_MIP',
 51: 'L_1',
 52: 'L_2',
 53: 'L_3a',
 54: 'L_6d',
 55: 'L_6mp',
 56: 'L_6v',
 57: 'L_p24pr',
 58: 'L_33pr',
 59: 'L_a24pr',
 60: 'L_p32pr',
 61: 'L_a24',
 62: 'L_d32',
 63: 'L_8BM',
 64: 'L_p32',
 65: 'L_10r',
 66: 'L_47m',
 67: 'L_8Av',
 68: 'L_8Ad',
 69: 'L_9m',
 70: 'L_8BL',
 71: 'L_9p',
 72: 'L_10d'